In [1]:
from io import BytesIO
from urllib.request import urlopen
import librosa
from transformers import Qwen2AudioForConditionalGeneration, Qwen2AudioProcessor
from ipywebrtc.webrtc import AudioRecorder

In [ ]:
model_path = "./models/Qwen_Qwen2-Audio-7B-Instruct"
processor = Qwen2AudioProcessor.from_pretrained(model_path)
model = Qwen2AudioForConditionalGeneration.from_pretrained(model_path, device_map="auto")

In [ ]:
recorder = AudioRecorder(recording=True, filename="recording", format="wav", autosave=True)
display(recorder)

In [ ]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/guess_age_gender.wav"},
    ]},
    {"role": "assistant", "content": "Yes, the speaker is female and in her twenties."},
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/translate_to_chinese.wav"},
    ]},
]

In [ ]:
text = processor.apply_chat_template(
    conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(
                    BytesIO(urlopen(ele['audio_url']).read()),
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True)
inputs.input_ids = inputs.input_ids.to("cuda")

generate_ids = model.generate(**inputs, max_length=256)
generate_ids = generate_ids[:, inputs.input_ids.size(1):]

response = processor.batch_decode(
    generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [4]:
def lol():
    return [
        {"role": "user", "content": [
            {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/guess_age_gender.wav"},
        ]},
        {"role": "assistant", "content": "Yes, the speaker is female and in her twenties."},
        {"role": "user", "content": [
            {"type": "audio", "audio_url": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2-Audio/audio/translate_to_chinese.wav"},
        ]},
]

In [ ]:
import requests
from io import BytesIO
from urllib.request import urlopen
from rich.pretty import pprint

conversation = lol()
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                file = BytesIO(urlopen(ele['audio_url']).read())
                files = {'file': file}
                upload_response = requests.post(
                    'http://localhost:3000/upload_audio/', files=files)
                audio_id = upload_response.json().get('audio_id')
                ele['audio_id'] = audio_id

# Generate response
generate_response = requests.post(
    'http://localhost:3000/generate', json=conversation)
pprint(generate_response.json())